In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


In [2]:
# 学習データ、テストデータの読み込み
train = pd.read_csv('/kaggle/input/titanic/train.csv')
test = pd.read_csv('/kaggle/input/titanic/test.csv')

# 学習データを特徴量と目的変数に分ける
train_X = train.drop(['Survived'], axis=1)
train_y = train['Survived']

# テストデータは特徴量のみなので、そのままで良い
test_X = test.copy()

In [3]:
from sklearn.preprocessing import LabelEncoder

# 変数PassengerIdを除外する
train_X = train_X.drop(['PassengerId'], axis=1)
test_X = test_X.drop(['PassengerId'], axis=1)

# 変数Name, Ticket, Cabinを除外する
train_X = train_X.drop(['Name', 'Ticket', 'Cabin'], axis=1)
test_X = test_X.drop(['Name', 'Ticket', 'Cabin'], axis=1)

# それぞれのカテゴリ変数にLabel Encodingを適用する
for c in ['Sex', 'Embarked']:
    le = LabelEncoder()
    train_X[c] = le.fit_transform(train_X[c].fillna('NA'))  # 学習データを変換
    test_X[c] = le.transform(test_X[c].fillna('NA'))  # テストデータを変換

# ここで、'Sex'と'Embarked'のカテゴリ変数が数値に変換されます


In [4]:
from xgboost import XGBClassifier  # クラス名を修正

# モデルの作成および学習データを与えての学習
model = XGBClassifier(n_estimators=20, random_state=71)  # XGBClassfier -> XGBClassifier
model.fit(train_X, train_y)

# テストデータの予測値を確率で出力する
pred = model.predict_proba(test_X)[:, 1]

# テストデータの予測値を二値に変換する
pred_label = np.where(pred > 0.5, 1, 0)

# 提出用ファイルの作成
submission = pd.DataFrame({'PassengerId': test['PassengerId'], 'Survived': pred_label})
submission.to_csv('submission_first.csv', index=False)

In [5]:
from sklearn.metrics import log_loss, accuracy_score
from sklearn.model_selection import KFold
from xgboost import XGBClassifier
import numpy as np

# 各foldのスコアを保存するリスト
scores_accuracy = []
scores_logloss = []

# クロスバリデーションを行う
# 学習データを４つに分割し、うち１つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
kf = KFold(n_splits=4, shuffle=True, random_state=71)
for tr_idx, va_idx in kf.split(train_X):  # コロンを追加
    # 学習デーやを学習データとバリデーションデータに分ける
    tr_X, va_X = train_X.iloc[tr_idx], train_X.iloc[va_idx]
    tr_y, va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx] 

    # モデルの学習を行う
    model = XGBClassifier(n_estimators=20, random_state=71)
    model.fit(tr_X, tr_y)

    # バリデーションデータの予測値を確率で出力する
    va_pred = model.predict_proba(va_X)[:, 1]

    # バリデーションデータでのスコアを計算する
    logloss = log_loss(va_y, va_pred)
    accuracy = accuracy_score(va_y, va_pred > 0.5)

    # そのfoldのスコアを保存する
    scores_logloss.append(logloss)
    scores_accuracy.append(accuracy)

# 各foldのスコアの平均を出力する
logloss = np.mean(scores_logloss)
accuracy = np.mean(scores_accuracy)
print(f'logloss:{logloss:.4f}, accuracy:{accuracy:.4f}')


logloss:0.4360, accuracy:0.8260


In [6]:
import itertools

# チューニング候補とするパラメータを準備する
param_space = {
    'max_depth':[3,5,7],
    'min_child_weight':[1.0,2.0,4.0]
}

# 探索するハイパーパラメータの組み合わせ
param_combinations = itertools.product(param_space['max_depth'],param_space['min_child_weight'])

# 各パラメータの組み合わせ、それに対するスコアを保存するリスト
params = []
scores = []

# 各パラメータの組み合わせごとに、クロスバリデーションで評価を行う
for max_depth,min_child_weight in param_combinations:
    score_folds = []
    # クロスバリデーションを行う
    # 学習データを４つに分割し、うち１つをバリデーションデータとすることを、バリデーションデータを変えて繰り返す
    kf = KFold(n_splits=4, shuffle=True, random_state=123456)
    for tr_idx, va_idx in kf.split(train_X):
        #学習データを学習データとバリデーションデータに分ける
        tr_X,va_X = train_X.iloc[tr_idx], train_X.iloc[va_idx]
        tr_y,va_y = train_y.iloc[tr_idx], train_y.iloc[va_idx]

        #モデルの学習を行う
        model = XGBClassifier(n_estimators=20, random_state=71,max_depth=max_depth,min_child_weight=min_child_weight)
        model.fit(tr_X,tr_y)

        # バリデーションデータでのスコアを計算し、保存する
        va_pred = model.predict_proba(va_X)[:,1]
        logloss = log_loss(va_y, va_pred)
        score_folds.append(logloss)

    # 各foldのスコアを平均する
    score_mean = np.mean(score_folds)

    # パラメータの組み合わせ、それに対するスコアを保存する
    params.append((max_depth, min_child_weight))
    scores.append(score_mean)

# 最もスコアが良いものをベストなパラメータとする
best_idx = np.argsort(scores)[0]
best_param = params[best_idx]
print(f'max_depth: {best_param[0]},min_child_weight:{best_param[1]}')

max_depth: 3,min_child_weight:4.0
